# Artifact - Account Detail - JSON Extraction and IPFS Integration

Date: 30AUG2024

** For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md **

1.1 -  Contract Deployment

In [31]:
from Crypto.Hash import keccak
import os
import binascii
from iroha import IrohaCrypto
from iroha import Iroha, IrohaGrpc
from iroha.ed25519 import H
import integration_helpers

from iroha.primitive_pb2 import can_set_my_account_detail
import sys

import csv

if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

IROHA_HOST_ADDR = os.getenv("IROHA_HOST_ADDR", "10.0.0.100")
IROHA_PORT = os.getenv("IROHA_PORT", "50051")
ADMIN_ACCOUNT_ID = os.getenv("ADMIN_ACCOUNT_ID", "admin@test")
ADMIN_PRIVATE_KEY = os.getenv(
    "ADMIN_PRIVATE_KEY",
    "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70",
)


user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key)
iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc("{}:{}".format(IROHA_HOST_ADDR, IROHA_PORT))

# Read account attributes from a csv
def read_accounts_from_csv(file_path):
    accounts = []
    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            accounts.append({
                'account_id': row['Account ID']
            })
    return accounts


# Path to the CSV file
csv_file_path = 'accounts.csv'

# Read accounts from CSV
accounts = read_accounts_from_csv(csv_file_path)

# Use the [n] account from the CSV for the example
account = accounts[4]

@integration_helpers.trace
def create_contract():
    bytecode = "608060405234801561001057600080fd5b5073a6abc17819738299b3b2c1ce46d55c74f04e290c6000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610b4c806100746000396000f3fe608060405234801561001057600080fd5b506004361061004c5760003560e01c80635bdb3a41146100515780637949a1b31461006f578063b7d66df71461009f578063d4e804ab146100cf575b600080fd5b6100596100ed565b6040516100669190610879565b60405180910390f35b61008960048036038101906100849190610627565b61024c565b6040516100969190610879565b60405180910390f35b6100b960048036038101906100b49190610693565b6103bb565b6040516100c69190610879565b60405180910390f35b6100d761059b565b6040516100e4919061085e565b60405180910390f35b606060006040516024016040516020818303038152906040527f5bdb3a41000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16836040516101be9190610830565b600060405180830381855af49150503d80600081146101f9576040519150601f19603f3d011682016040523d82523d6000602084013e6101fe565b606091505b509150915081610243576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161023a9061091e565b60405180910390fd5b80935050505090565b60606000838360405160240161026392919061089b565b6040516020818303038152906040527f7949a1b3000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168360405161032a9190610830565b600060405180830381855af49150503d8060008114610365576040519150601f19603f3d011682016040523d82523d6000602084013e61036a565b606091505b5091509150816103af576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016103a69061091e565b60405180910390fd5b80935050505092915050565b606060008484846040516024016103d4939291906108d2565b6040516020818303038152906040527fb7d66df7000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168360405161049b9190610830565b600060405180830381855af49150503d80600081146104d6576040519150601f19603f3d011682016040523d82523d6000602084013e6104db565b606091505b509150915081610520576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016105179061091e565b60405180910390fd5b8460405161052e9190610847565b6040518091039020866040516105449190610847565b60405180910390208860405161055a9190610847565b60405180910390207f5e1b38cd47cf21b75d5051af29fa321eedd94877db5ac62067a076770eddc9d060405160405180910390a48093505050509392505050565b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b60006105d26105cd84610963565b61093e565b9050828152602081018484840111156105ea57600080fd5b6105f5848285610a14565b509392505050565b600082601f83011261060e57600080fd5b813561061e8482602086016105bf565b91505092915050565b6000806040838503121561063a57600080fd5b600083013567ffffffffffffffff81111561065457600080fd5b610660858286016105fd565b925050602083013567ffffffffffffffff81111561067d57600080fd5b610689858286016105fd565b9150509250929050565b6000806000606084860312156106a857600080fd5b600084013567ffffffffffffffff8111156106c257600080fd5b6106ce868287016105fd565b935050602084013567ffffffffffffffff8111156106eb57600080fd5b6106f7868287016105fd565b925050604084013567ffffffffffffffff81111561071457600080fd5b610720868287016105fd565b9150509250925092565b610733816109e2565b82525050565b600061074482610994565b61074e81856109aa565b935061075e818560208601610a23565b61076781610ab6565b840191505092915050565b600061077d82610994565b61078781856109bb565b9350610797818560208601610a23565b80840191505092915050565b60006107ae8261099f565b6107b881856109c6565b93506107c8818560208601610a23565b6107d181610ab6565b840191505092915050565b60006107e78261099f565b6107f181856109d7565b9350610801818560208601610a23565b80840191505092915050565b600061081a6027836109c6565b915061082582610ac7565b604082019050919050565b600061083c8284610772565b915081905092915050565b600061085382846107dc565b915081905092915050565b6000602082019050610873600083018461072a565b92915050565b600060208201905081810360008301526108938184610739565b905092915050565b600060408201905081810360008301526108b581856107a3565b905081810360208301526108c981846107a3565b90509392505050565b600060608201905081810360008301526108ec81866107a3565b9050818103602083015261090081856107a3565b9050818103604083015261091481846107a3565b9050949350505050565b600060208201905081810360008301526109378161080d565b9050919050565b6000610948610959565b90506109548282610a56565b919050565b6000604051905090565b600067ffffffffffffffff82111561097e5761097d610a87565b5b61098782610ab6565b9050602081019050919050565b600081519050919050565b600081519050919050565b600082825260208201905092915050565b600081905092915050565b600082825260208201905092915050565b600081905092915050565b60006109ed826109f4565b9050919050565b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b82818337600083830152505050565b60005b83811015610a41578082015181840152602081019050610a26565b83811115610a50576000848401525b50505050565b610a5f82610ab6565b810181811067ffffffffffffffff82111715610a7e57610a7d610a87565b5b80604052505050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b6000601f19601f8301169050919050565b7f4572726f722063616c6c696e67207365727669636520636f6e7472616374206660008201527f756e6374696f6e0000000000000000000000000000000000000000000000000060208201525056fea26469706673582212206ad40afbd4cc9c87ae154542d003c9538e4b89473a13cadd3cbf618ea181206864736f6c63430008040033"
    """Bytecode was generated using remix editor  https://remix.ethereum.org/ from file detail.sol. """
    tx = iroha.transaction(
        [iroha.command("CallEngine", caller=ADMIN_ACCOUNT_ID, input=bytecode)]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    net.send_tx(tx)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    for status in net.tx_status_stream(tx):
        print(status)
    return hex_hash


@integration_helpers.trace
def set_account_detail(address, account, variable_1, variable_2):
    params = integration_helpers.get_first_four_bytes_of_keccak(
        b"setAccountDetail(string,string,string)"
    )
    no_of_param = 3
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(
            x, no_of_param
        )
    params = params + integration_helpers.argument_encoding(
        account['account_id']
    )  # source account id
    params = params + integration_helpers.argument_encoding(variable_1)  # key
    params = params + integration_helpers.argument_encoding(variable_2)  #  value
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    print(response)
    for status in net.tx_status_stream(tx):
        print(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash


@integration_helpers.trace
def get_account_details():
    params = integration_helpers.get_first_four_bytes_of_keccak(b"getAccountDetail()")
    no_of_param = 0
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash


hash = create_contract()
address = integration_helpers.get_engine_receipts_address(hash)
# hash = get_account_details()
integration_helpers.get_engine_receipts_result(hash)
# hash = set_account_detail(address, account)
# print(account)
# hash = get_account_details()
# integration_helpers.get_engine_receipts_result(hash)
print("done")


	Entering "create_contract"
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "create_contract"
	Entering "get_engine_receipts_address"
	Leaving "get_engine_receipts_address"
	Entering "get_engine_receipts_result"

	Leaving "get_engine_receipts_result"
done


1.2 -  Account Detail Setting

Extracts metadata info from 'assets.json' and injects it as account details, calling function `set_account_details`

In [33]:
from json_ipfs_functions import *

def main():
    file_path = os.path.join('upload', 'assets.json')
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    else:
        
        result = process_json_data(data)
        
        metadata_cids = []
        file_cids = []

        for i in range(len(result['file_cids'])):
            file_cid = result['file_cids'][i]["cid"]
            json_cid_value = result['file_cids'][i]["json_cid_value"]

            metadata_cids.append(json_cid_value)
            file_cids.append(file_cid)

        metadata_index = 0
        file_index = 0

        while metadata_index < len(metadata_cids) and file_index < len(file_cids):
            variable_1 = f"file_{metadata_index+1}_metadata_CID"
            variable_2 = metadata_cids[metadata_index]
            print(f"{variable_1} = {variable_2}")
            metadata_index += 1
            hash = set_account_detail(address, account, variable_1, variable_2)
            print(account)

            if metadata_index <= len(metadata_cids) and file_index < len(file_cids):
                variable_1 = f"file_{metadata_index}_CID"
                variable_2 = file_cids[file_index]
                print(f"{variable_1} = {variable_2}")
                file_index += 1
                hash = set_account_detail(address, account, variable_1, variable_2)
                print(account)

if __name__ == '__main__':
    main()

file_1_metadata_CID = QmfJmXYxRGJXSbCXgHJrnQWVXKRsxJ2sAubPwU8iZf68ca
	Entering "set_account_detail"
None
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
{'account_id': 'nice_moore@test'}
file_1_CID = QmeBTz4ZwyqPkPigpASZ3wJodBXzJRpeXQMwNcsWRqU5Jv
	Entering "set_account_detail"
None
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
{'account_id': 'nice_moore@test'}
file_2_metadata_CID = QmNPnr5bUAt4WKC4Rgh4w5gZfwmHAEdU6a24qehjBxXkCC
	Entering "set_account_detail"
None
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
{'account_id': 'nice_moore@test'}
file_2_CID = QmSSY49SnmbCZ3oSaTki7CYZe1ZaWZfE1CsWHpt8Ge7acJ
	Entering "set_account_detail"
None
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_

1.3 - Queries the account and checks the proper setting of details.

In [34]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=account['account_id'])
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
print(f'Account id = {account}, details = {data.detail}')

Account id = {'account_id': 'nice_moore@test'}, details = { "admin@test" : { "file_1_CID" : "QmeBTz4ZwyqPkPigpASZ3wJodBXzJRpeXQMwNcsWRqU5Jv", "file_1_metadata_CID" : "QmfJmXYxRGJXSbCXgHJrnQWVXKRsxJ2sAubPwU8iZf68ca", "file_2_CID" : "QmSSY49SnmbCZ3oSaTki7CYZe1ZaWZfE1CsWHpt8Ge7acJ", "file_2_metadata_CID" : "QmNPnr5bUAt4WKC4Rgh4w5gZfwmHAEdU6a24qehjBxXkCC", "file_3_CID" : "QmRA3NWM82ZGynMbYzAgYTSXCVM14Wx1RZ8fKP42G6gjgj", "file_3_metadata_CID" : "QmUMn338ahX7vynkMyRz5H4KdQe2HDZ4y6LnHHG591WuhC", "file_4_CID" : "QmVTTcqbGvYRn7n7uPYwk4vi8NdbeYvnZkX9MVT3byrAx2", "file_4_metadata_CID" : "QmXa3ZiZBHtgptXz161yHiPxcT6xRnJ6dku5SVtrSqvDpA", "file_5_CID" : "QmdiRawzVNUiB28ENKQ7WefeFLEJ1xMjsJjwtHL2jnJ9xW", "file_5_metadata_CID" : "QmUSDqEVZ7pkHDzY8BMA83YzFgnDo7Nnf6VeqaUU579gDd" } }


1.4 - Queries the account above and extract details to download files and metadatada

In [35]:
#Working version 24AUG2024
from json_ipfs_functions import *
import json
from concurrent.futures import ThreadPoolExecutor

# # Define the base directory for downloading files
BASE_DOWNLOAD_DIR = "download"

def get_output_path(file_name):
    """Returns the output path for downloading a file to the 'download' directory."""
    return f"{BASE_DOWNLOAD_DIR}/{original_file_name}"

# def download_file_from_ipfs(cid):
#     try:
#         # Download the file from IPFS using the provided CID
#         downloaded_file = ipfs.get(cid)
#         return downloaded_file.path
#     except Exception as e:
#         ic(f"Failed to download file: {str(e)}")
#         return None

def main():
    # 1 - Parse Script #1 Output and retrieve all files metadata
    accounts = json.loads(data.detail)  # Since 'admin@test' is the top-level key, we'll consider it as an account
    ic(accounts)
    

    for account in accounts.values():
        ic(account)
        
        # Extract file metadata from the JSON data
        files_metadata = {}
        for key, value in account.items():
            if '_CID' in key:
                cid_key = key.replace('_CID', '')
                if f'{cid_key}_metadata_CID' in account.keys():
                    cid_value = value
                    metadata_cid_value = account[f'{cid_key}_metadata_CID']
                    files_metadata[cid_key] = {'cid': cid_value, 'metadata_cid': metadata_cid_value}

        # ic("Files metadata:")
        for file_name, file_data in files_metadata.items():
            ic(f"  - {file_name}:")
            ic(f"    CID: {file_data['cid']}")
            ic(f"    Metadata CID: {file_data['metadata_cid']}")

            # Call `download_json_from_ipfs` function and retrieves the actual json metadata
            try:
                file_metadata_json = download_json_from_ipfs(file_data['metadata_cid'])
                ic("File metadata:")
                # ic(json.dumps(file_metadata_json, indent=4))

                # From the json key for file_name extracts its value
                original_file_name = file_metadata_json.get('file_name', 'Unknown')
                ic("aqui_1 : ", original_file_name)
                # output_path = get_output_path(original_file_name)
            except Exception as e:
                ic(f"Failed to download metadata JSON for file '{file_name}': {str(e)}")

        # Download files from IPFS using their CID
        with ThreadPoolExecutor() as executor:
            tasks = []
            for file_name, file_data in files_metadata.items():
                cid = file_data['cid']
                ic(file_name)
                ic(cid)
                ic("aqui_2 : ", original_file_name)

                # Assuming that each file has a corresponding metadata json
                original_file_name = file_data.get('file_name', 'Unknown')
                output_path = f"download/{original_file_name}"
                ic(output_path)
        
                task = executor.submit(download_file_from_ipfs, cid, output_path)
                tasks.append((file_name, task))

if __name__ == '__main__':
    main()

ic| accounts: {'admin@test': {'file_1_CID': 'QmeBTz4ZwyqPkPigpASZ3wJodBXzJRpeXQMwNcsWRqU5Jv',
                              'file_1_metadata_CID': 'QmfJmXYxRGJXSbCXgHJrnQWVXKRsxJ2sAubPwU8iZf68ca',
                              'file_2_CID': 'QmSSY49SnmbCZ3oSaTki7CYZe1ZaWZfE1CsWHpt8Ge7acJ',
                              'file_2_metadata_CID': 'QmNPnr5bUAt4WKC4Rgh4w5gZfwmHAEdU6a24qehjBxXkCC',
                              'file_3_CID': 'QmRA3NWM82ZGynMbYzAgYTSXCVM14Wx1RZ8fKP42G6gjgj',
                              'file_3_metadata_CID': 'QmUMn338ahX7vynkMyRz5H4KdQe2HDZ4y6LnHHG591WuhC',
                              'file_4_CID': 'QmVTTcqbGvYRn7n7uPYwk4vi8NdbeYvnZkX9MVT3byrAx2',
                              'file_4_metadata_CID': 'QmXa3ZiZBHtgptXz161yHiPxcT6xRnJ6dku5SVtrSqvDpA',
                              'file_5_CID': 'QmdiRawzVNUiB28ENKQ7WefeFLEJ1xMjsJjwtHL2jnJ9xW',
                              'file_5_metadata_CID': 'QmUSDqEVZ7pkHDzY8BMA83YzFgnDo7Nnf6VeqaUU579gDd'}}
ic| account: {

In [44]:
import json
import requests
from json_ipfs_functions import *

def main():
        # Load JSON metadata from the provided string
        metadata = json.loads(data.detail)
        ic(metadata)

        # Iterate over each file in the metadata
        for username, files in metadata.items():
            for file_id, cids in files.items():
                if file_id.endswith('_CID'):
                    # Skip CID fields, we're interested in metadata_CID and file_CID
                    continue

                if file_id.endswith('_metadata_CID'):
                    # Get the file's metadata by fetching its JSON from IPFS
                    metadata_cid = cids
                    response = download_json_from_ipfs('{metadata_cid}')
                    metadata_json = json.loads(response.text)

                    # Extract the original file name (file_name) from the metadata
                    original_file_name = metadata_json['file_name']
                    ic(original_file_name)

                if file_id.endswith('_CID'):
                    # Get the actual file by downloading it from IPFS
                    file_cid = cids
                    response = download_file_from_ipfs('{file_cid}')
                    with open(original_file_name, 'wb') as f:
                        f.write(response.content)

if __name__ == '__main__':
    main()

ic| metadata: {'admin@test': {'file_1_CID': 'QmeBTz4ZwyqPkPigpASZ3wJodBXzJRpeXQMwNcsWRqU5Jv',
                              'file_1_metadata_CID': 'QmfJmXYxRGJXSbCXgHJrnQWVXKRsxJ2sAubPwU8iZf68ca',
                              'file_2_CID': 'QmSSY49SnmbCZ3oSaTki7CYZe1ZaWZfE1CsWHpt8Ge7acJ',
                              'file_2_metadata_CID': 'QmNPnr5bUAt4WKC4Rgh4w5gZfwmHAEdU6a24qehjBxXkCC',
                              'file_3_CID': 'QmRA3NWM82ZGynMbYzAgYTSXCVM14Wx1RZ8fKP42G6gjgj',
                              'file_3_metadata_CID': 'QmUMn338ahX7vynkMyRz5H4KdQe2HDZ4y6LnHHG591WuhC',
                              'file_4_CID': 'QmVTTcqbGvYRn7n7uPYwk4vi8NdbeYvnZkX9MVT3byrAx2',
                              'file_4_metadata_CID': 'QmXa3ZiZBHtgptXz161yHiPxcT6xRnJ6dku5SVtrSqvDpA',
                              'file_5_CID': 'QmdiRawzVNUiB28ENKQ7WefeFLEJ1xMjsJjwtHL2jnJ9xW',
                              'file_5_metadata_CID': 'QmUSDqEVZ7pkHDzY8BMA83YzFgnDo7Nnf6VeqaUU579gDd'}}


In [47]:
# main_script.py

from json_ipfs_functions import *
import json

def main():
        # Load JSON metadata from the provided string
        metadata = json.loads(data.detail)
        ic(metadata)


        processed_result = {}
        for username, files in metadata.items():
            processed_files = process_json_data(files)
            processed_result[username] = {'file_cids': [], 'json_cids': {}}
            for file_n_metadata_cid_value in processed_files['json_cids'].keys():
                cid = [f['cid'] for f in processed_files['file_cids'] if f['json_cid_value'] == file_n_metadata_cid_value][0]
                original_file_name = file_n_metadata_cid_value.split("_")[0]
        
                metadata_json = download_json_from_ipfs(file_n_metadata_cid_value)
        
                if metadata_json:
                    file_name = json.loads(metadata_json)['file_name']
            
                    download_file_from_ipfs(cid, "downloaded_" + file_name)

                    print(f"File downloaded successfully: {file_name}")